In [ ]:
import numpy as np
import random
from shapely.geometry import Point, MultiPoint
from scipy.optimize import differential_evolution as scipy_de
from shapely.ops import nearest_points

import voronoi_utils as vu
import solvers

def is_feasible(n, fixed, polygon, A_min, params, solver="custom"):
    if n == 0:
        _, areas = vu.get_voronoi_cells_and_areas(fixed, polygon)
        ok = min(areas) >= A_min
        print(f"[n=0] {'feasible' if ok else 'not feasible'}")
        if ok:
            vu.plot_voronoi(fixed, [], polygon)
        return ok, []

    minx, miny, maxx, maxy = polygon.bounds
    bounds = [(minx, maxx), (miny, maxy)] * n

    def loss(x_flat):
        pts = [(x_flat[i], x_flat[i+1]) for i in range(0, len(x_flat), 2)]
        for xi, yi in pts:
            if not polygon.contains(Point(xi, yi)):
                return 1e6
        _, areas = vu.get_voronoi_cells_and_areas(fixed + pts, polygon)
        return -min(areas)

    if solver == "custom":
        bx, bf = solvers.differential_evolution_custom(loss, bounds, polygon, A_min, params)
    elif solver == "scipy":
        # Early‐exit callback: stop if the minimum Voronoi cell area ≥ A_min
        def _callback(xk, convergence=None):
            # xk is the current best candidate vector
            # loss(xk) = –(min cell area), so –loss(xk) == min cell area
            if -loss(xk) >= A_min:
                return True  # tells SciPy to terminate early
        res = scipy_de(
            loss, bounds,
            maxiter=params["maxiter"],
            popsize=params["popsize"],
            seed=params.get("seed", None),
            updating="deferred",
            callback=_callback
        )
        bx, bf = res.x, res.fun

    elif solver == "qpso":
        bx, bf = solvers.qpso_solver(loss, bounds, polygon, A_min, params)
    elif solver == "qpso2":
        bx, bf = solvers.qpso_pairwise_solver(loss, bounds, polygon, A_min, params)
    elif solver == "ga":
        bx, bf = solvers.ga_solver(loss, bounds, polygon, A_min, params)
    elif solver == "spsa":
        bx, bf = solvers.spsa_solver(loss, bounds, polygon, A_min, params)
    else:
        raise ValueError("Unknown solver")

    best_min = -bf
    ok = best_min >= A_min
    print(f"[n={n}] {'feasible' if ok else 'not feasible'} with min_cell_area = {best_min:.4f}")
    if ok:
        added = [(bx[i], bx[i+1]) for i in range(0, len(bx), 2)]
        vu.plot_voronoi(fixed, added, polygon)
        return True, added
    return False, []

def find_max_additional(fixed, polygon, A_min, params, solver="custom"):
    print(f"Minimum area threshold (A_min): {A_min:.4f}")
    ok, _ = is_feasible(0, fixed, polygon, A_min, params, solver)
    if not ok:
        return 0, []

    low, n = 0, 1
    best_added = []
    while True:
        ok, added = is_feasible(n, fixed, polygon, A_min, params, solver)
        if not ok:
            break
        low, best_added = n, added
        n *= 2

    print(f"Starting binary search between n={low} (feasible) and n={n} (infeasible)")
    lo, hi = low, n
    while lo < hi - 1:
        mid = (lo + hi) // 2
        ok, added = is_feasible(mid, fixed, polygon, A_min, params, solver)
        if ok:
            lo, best_added = mid, added
        else:
            hi = mid

    print(f"Result → Max extra points: {lo}")
    return lo, best_added


# seed
 = 3 we have 8 achivealbe
 
 = 30 we have 9

 = 6 we have 8
 = 8 we have 9

In [ ]:
s = 8
random.seed(s)
np.random.seed(s)

pts = np.random.rand(30, 2)
polygon = MultiPoint([tuple(p) for p in pts]).convex_hull

fixed = []
minx, miny, maxx, maxy = polygon.bounds
while len(fixed) < 5:
    x, y = random.uniform(minx, maxx), random.uniform(miny, maxy)
    if polygon.contains(Point(x, y)):
        fixed.append((x, y))

A_min = polygon.area / 15


spsa_params   = {"maxiter": 300, "a": 0.1, "c": 0.001,
                    "alpha": 0.5, "gamma": 0.001, "restarts": 30, "seed": None}
custom_params = {"maxiter": 400, "popsize": 200, "F": 0.8, "CR": 0.7, "seed": None}
scipy_params  = {"maxiter": 50, "popsize": 25, "seed": None}
qpso_params   = {"maxiter": 350, "popsize": 50, "alpha": 0.75, "seed": None}
ga_params     = {
    "popsize": 50,
    "ngen": 350,
    "cxpb": 0.6,
    "mutpb": 0.3,
    "sigma": 0.1,
    "indpb": 0.1,
    "seed": None
}

In [ ]:
print("=== SPSA Solver ===")
find_max_additional(fixed, polygon, A_min, spsa_params, solver="spsa")

In [ ]:
print("=== Custom DE ===")
find_max_additional(fixed, polygon, A_min, custom_params, solver="custom")

In [ ]:
print("\n=== SciPy DE ===")
find_max_additional(fixed, polygon, A_min, scipy_params, solver="scipy")

In [ ]:
print("\n=== QPSO (original) ===")
find_max_additional(fixed, polygon, A_min, qpso_params, solver="qpso")

In [ ]:
print("\n=== QPSO (pairwise) ===")
find_max_additional(fixed, polygon, A_min, qpso_params, solver="qpso2")

In [ ]:
print("\n=== GA Solver ===")
find_max_additional(fixed, polygon, A_min, ga_params, solver="ga")